# Baseten custom pipeline use case

Let's assume that we want to upload the following pipeline to be served by baseten. Some differences with the standard use case are:
- It is a scikit-learn pipeline instead of a simple scikit-learn model
- Basides 3rd party libraries that could be install via a `requirements.txt` file, it has also local dependencies

In [1]:
!pip install truss==0.0.30 baseten==0.2.1

0.1.28


0.0.20


In [1]:
import numpy as np
import sys
sys.path.insert(0, './pipelines/lr_model/')
sys.path.insert(0, './models/')
from joblib import dump
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from pipelines.lr_model.steps import (Disqualify, FeatureCalculator)

LEARNED_INTERCEPT = -1
LEARNED_COEFFICIENTS = [2, 1, 0, 0, 1]
lr_model = LogisticRegression()
lr_model.classes_ = np.array([0, 1])
lr_model.intercept_ = LEARNED_INTERCEPT
lr_model.coef_ = np.array([LEARNED_COEFFICIENTS])

pipeline = Pipeline(
    steps=[
        ("step1", Disqualify()),
        (
            "step2",
            FeatureCalculator(check_qualified=True),
        ),
        ("model", lr_model),
    ]
)

from bson.objectid import ObjectId

SID0 = ObjectId("abcdef123456abcdef123453")
SID1 = ObjectId("abcdef123456abcdef123453")
SID2 = ObjectId("abcdef123456abcdef123454")
SID3 = ObjectId("abcdef123456abcdef123454")
SID4 = ObjectId("abcdef123456abcdef543210")

BUILDER = {
    "feature1": SID0,
    "feature2": SID1,
    "feature3": SID4,
    "feature4": SID2,
    "feature5": SID1,
}

ROLE = {
    "feature1": SID0,
    "feature2": SID1,
    "feature3": SID1,
    "feature4": SID3,
    "feature5": SID4,
}

pipeline.predict_proba([(x, ROLE) for x in [BUILDER]])

# somehow I should upload this pipeline and the necessary local dependencies to baseten

array([[0.11920292, 0.88079708]])

## Package Model

In [2]:
import truss
handle = truss.mk_truss(pipeline, target_directory="./pipeline_truss/", requirements_file="./requirements.txt", bundled_packages=["./models/", "./pipelines/"])

/Users/bola/repos/a.team/baseten-demo/.env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Test Locally

In [3]:
handle.server_predict({'inputs': list([(x, ROLE) for x in [BUILDER]])})

{'predictions': array([1]),
 'probabilities': [[0.11920292202211769, 0.8807970779778823]]}

## Deploy to Baseten

In [4]:
import baseten
baseten.login("")

INFO API key set.


In [5]:
b10_model = baseten.deploy_truss(handle, model_name='test-truss-old')

INFO Serializing test-truss-old truss.
INFO Making contact with BaseTen 👋 👽
INFO 🚀 Uploading model to BaseTen 🚀
Upload Progress: 100% |████████████████████████████████████████████████████████████████████████████████████████████████████| 7.22k/7.22k
INFO 🔮 Upload successful!🔮
INFO eyJtb2RlbF90eXBlIjogIk1vZGVsIiwgIm1vZGVsX2ZyYW1ld29yayI6ICJza2xlYXJuIiwgIm1vZGVsX21vZHVsZV9kaXIiOiAibW9kZWwiLCAibW9kZWxfY2xhc3NfZmlsZW5hbWUiOiAibW9kZWwucHkiLCAibW9kZWxfY2xhc3NfbmFtZSI6ICJNb2RlbCIsICJkYXRhX2RpciI6ICJkYXRhIiwgImlucHV0X3R5cGUiOiAiQW55IiwgIm1vZGVsX21ldGFkYXRhIjogeyJtb2RlbF9iaW5hcnlfZGlyIjogIm1vZGVsIiwgInN1cHBvcnRzX3ByZWRpY3RfcHJvYmEiOiB0cnVlfSwgInJlcXVpcmVtZW50cyI6IFsiYXBwbm9wZT09MC4xLjMiLCAiYXN0dG9rZW5zPT0yLjAuNSIsICJiYWNrY2FsbD09MC4yLjAiLCAiYmFzZXRlbj09MC4yLjEiLCAiYnNvbj09MC41LjEwIiwgImNlcnRpZmk9PTIwMjIuNi4xNSIsICJjaGFyc2V0LW5vcm1hbGl6ZXI9PTIuMS4wIiwgImNsaWNrPT04LjEuMyIsICJjb2xvcmFtYT09MC40LjUiLCAiY29vbG5hbWU9PTEuMS4wIiwgImRlYnVncHk9PTEuNi4yIiwgImRlY29yYXRvcj09NS4xLjEiLCAiZW50cnlwb2ludHM9PTAuNCI

In [10]:
import json
from bson import ObjectId

class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return json.JSONEncoder.default(self, o)

request = {'role': ROLE, 'builder': BUILDER}
serial_req = JSONEncoder().encode(request)

b10_model.predict([serial_req], prediction_only=False)

ApiError: Error calling predict.
<Server response: b'{"error": "Failed to invoke model version yqvvpxq. \\nTraceback (most recent call last):\\n  File \\"/app/model_wrapper.py\\", line 55, in predict\\n    return self._model.predict(request)\\n  File \\"/app/model/model.py\\", line 41, in predict\\n    result = self._model.predict(inputs)\\n  File \\"/usr/local/lib/python3.9/site-packages/sklearn/pipeline.py\\", line 457, in predict\\n    Xt = transform.transform(Xt)\\n  File \\"/packages/pipelines/lr_model/steps.py\\", line 14, in transform\\n    return [self._disqualify(*x) for x in X]\\n  File \\"/packages/pipelines/lr_model/steps.py\\", line 14, in <listcomp>\\n    return [self._disqualify(*x) for x in X]\\nTypeError: _disqualify() takes 2 positional arguments but 423 were given\\n"}'>